# Common Data Fields
This notebook will walk through how to access some of the more common data fields from the SRI hdf5 data files.  This should be treated as a quick reference for common scientific applications and is NOT intended to be a comprehensive map of everything in these files.  Refer to metadata contained within the files for descriptions of other arrays.

In [1]:
import h5py

In [2]:
filename = 'data/20200207.001_lp_5min-fitcal.h5'

h5 = h5py.File(filename, 'r')

Throughout this notebook, the expected dimensions of each data field will be listed using the following definitions:
- Nrecords: Number of time records
- Nbeams: Number of radar beams
- Nranges: Number of radar range gates

## Electron Density

**Units:** m$^{-3}$

**Dimensions:** Nrecords x Nbeams x Nranges

In [3]:
ne = h5['FittedParams/Ne'][:]
print(ne.shape)

(188, 11, 74)


## Ion Temperature
The fourth index corresponds to the ion species (0 = O+).

**Units:** K

**Dimensions:** Nrecords x Nbeams x Nranges

In [4]:
Ti = h5['FittedParams/Fits'][:,:,:,0,1]
print(Ti.shape)

(188, 11, 74)


## Electron Temperature

**Units:** K

**Dimensions:** Nrecords x Nbeams x Nranges

In [5]:
Te = h5['FittedParams/Fits'][:,:,:,-1,1]
print(Te.shape)

(188, 11, 74)


## Line-of-Sight Velocity

**Units:** m/s

**Dimensions:** Nrecords x Nbeams x Nranges

In [6]:
Vlos = h5['FittedParams/Fits'][:,:,:,-1,3]
print(Vlos.shape)

(188, 11, 74)


## Time
The `Time` group lists timestamps in a variety of formats, but `UnixTime` (seconds from January 1, 1970) is often most convenient for programing purposes.

**Dimensions:** Nrecords x 2

In [7]:
utime = h5['Time/UnixTime'][:]
print(utime.shape)

(188, 2)


Note the two columns in this array correspond to the start and end of each integration period.  If you need a 1D array of time stamps (common for most plotting and basic analysis), it is generally acceptable to take either just the first column (start times) or the midpoint of the interval.

In [8]:
# start times
utime_1d = utime[:,0]

# midpoints
import numpy as np
utime_1d = np.mean(utime, axis=1)

To convert these unix timestamps to datetime objects, use either the native python [datetime](https://docs.python.org/3/library/datetime.html) library or the [numpy datetime](https://numpy.org/doc/stable/reference/arrays.datetime.html) functionality.

In [9]:
import datetime as dt
import numpy as np

# native python datetime library
time = [dt.datetime.utcfromtimestamp(ut) for ut in utime_1d]

# numpy datetime
time = utime_1d.astype('datetime64[s]')

## Geographic Position
Geodetic latitude, longitude, and altitude of the center of each radar bin.

**Dimensions:** Nbeams x Nranges

In [10]:
glat = h5['Geomag/Latitude'][:]
glon = h5['Geomag/Longitude'][:]
galt = h5['Geomag/Altitude'][:]

print(glat.shape, glon.shape, galt.shape)

(11, 74) (11, 74) (11, 74)


## Geomagnetic Position
Geomagnetic latitude and longitude of the center of each radar bin.  If your application is dependent on highly accurate magnetic position relative to some outside dataset, it is recommended that you start with the geographic postion and calculate the conversion to magnetic coordinates yourself to avoid ambiguity with different magnetic coordinate systems and IGRF versions.

**Dimensions:** Nbeams x Nranges

In [11]:
mlat = h5['Geomag/MagneticLatitude'][:]
mlon = h5['Geomag/MagneticLongitude'][:]

print(mlat.shape, mlon.shape)

(11, 74) (11, 74)


In [12]:
# Close h5 object
h5.close()